In [2]:
import os
import time
import pandas as pd
import numpy as np
from geojson import dump
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import networkx as nx
import os
import osmnx as ox
import pickle
import struct
import csv

In [21]:
def loadQUANTMatrix(filename):
    with open(filename,'rb') as f:
        (m,) = struct.unpack('i', f.read(4))
        (n,) = struct.unpack('i', f.read(4))
        print("loadQUANTMatrix::m=",m,"n=",n)
        matrix = np.arange(m*n,dtype=np.float64).reshape(m, n) #and hopefully m===n, but I'm not relying on it
        for i in range(0,m):
            data = struct.unpack('{0}f'.format(n), f.read(4*n)) #read a row back from the stream - data=list of floats
            for j in range(0,n):
                matrix[i,j] = data[j]
    return matrix

def saveMatrix(matrix,filename):
    with open(filename,'wb') as f:
        pickle.dump(matrix,f)

class QUANTLHModel:
    """
    constructor
    @param n number of residential zones (MSOA)
    @param m number of school point zones
    """

    def __init__(self, m, n):
        # constructor
        self.m = m
        self.n = n
        self.Ei = np.zeros(m)
        self.Aj = np.zeros(n)
        self.cij_0 = np.zeros(1)  # costs matrix - set to something BIG later
        self.cij_1 = np.zeros(1)  # costs matrix - set to something BIG later
        self.cij_2 = np.zeros(1)  # costs matrix - set to something BIG later
        self.SObs_0 = np.zeros(1)
        self.SObs_1 = np.zeros(1)
        self.SObs_2 = np.zeros(1)

    ################################################################################

    """
    setPopulationVectorEi
    Overload of setPopulationEi to set Ei directly from a vector, rather than a Pandas dataframe.
    """
    def setPopulationVectorEi(self, Ei):
        self.Ei = Ei
        assert len(self.Ei) == self.m, "FATAL: setPopulationEi length Ei=" + str(
            len(self.Ei)) + " MUST equal model definition size of m=" + str(self.m)

    ################################################################################

    """
    setPopulationEi
    Given a data frame containing one column with the zone number (i) and one column
    with the actual data values, fill the Ei population property with data so that
    the position in the Ei numpy array is the zonei field of the data.
    The code is almost identical to the setAttractorsAj method.
    NOTE: max(i) < self.m
    """
    def setPopulationEi(self, df, zoneiColName, dataColName):
        df2 = df.sort_values(by=[zoneiColName])
        self.Ei = df2[dataColName].to_numpy()
        assert len(self.Ei) == self.m, "FATAL: setPopulationEi length Ei=" + str(len(self.Ei)) + " MUST equal model definition size of m=" + str(self.m)

    ################################################################################

    """
    setAttractorsAj
    Given a data frame containing one column with the zone number (j) and one column
    with the actual data values, fill the Aj attractors property with data so that
    the position in the Aj numpy array is the zonej field of the data.
    The code is almost identical to the setPopulationEi method.
    NOTE: max(j) < self.n
    """
    def setAttractorsAj(self, df, zonejColName, dataColName):
        df2 = df.sort_values(by=[zonejColName])
        self.Aj = df2[dataColName].to_numpy()
        assert len(self.Aj) == self.n, "FATAL: setAttractorsAj length Aj=" + str(len(self.Aj)) + " MUST equal model definition size of n=" + str(self.n)

    ################################################################################

    """
    setCostsMatrix
    Assign the cost matrix for the model to use when it runs.
    NOTE: this MUST match the m x n order of the model and be a numpy array
    """
    def setCostMatrixCij(self, cij_0, cij_1, cij_2):
        i, j = cij_0.shape  # Hopefully cij_0.shape = cij_1.shape = cij_2.shape
        assert i == self.m, "FATAL: setCostsMatrix cij matrix is the wrong size, cij.m=" + str(i) + " MUST match model definition of m=" + str(self.m)
        assert j == self.n, "FATAL: setCostsMatrix cij matrix is the wrong size, cij.n=" + str(j) + " MUST match model definition of n=" + str(self.n)
        self.cij_0 = cij_0
        self.cij_1 = cij_1
        self.cij_2 = cij_2

    ################################################################################

    """
    setObsMatrix
    Assign the cost matrix for the model to use when it runs.
    NOTE: this MUST match the m x n order of the model and be a numpy array
    """
    def setObsMatrix(self, SObs_0, SObs_1, SObs_2):
        i, j = SObs_0.shape  # Hopefully cij_0.shape = SObs_0.shape = SObs_0.shape
        assert i == self.m, "FATAL: setCostsMatrix cij matrix is the wrong size, cij.m=" + str(
            i) + " MUST match model definition of m=" + str(self.m)
        assert j == self.n, "FATAL: setCostsMatrix cij matrix is the wrong size, cij.n=" + str(
            j) + " MUST match model definition of n=" + str(self.n)
        self.SObs_0 = SObs_0
        self.SObs_1 = SObs_1
        self.SObs_2 = SObs_2

    ################################################################################

    """
    computeCBar
    Compute average trip length TODO: VERY COMPUTATIONALLY INTENSIVE - FIX IT
    @param Sij trips matrix containing the flow numbers between MSOA (i) and schools (j)
    @param cij trip times between i and j
    """
    def computeCBar(self, Sij, cij):
        CNumerator = np.sum(Sij * cij)
        CDenominator = np.sum(Sij)
        cbar = CNumerator / CDenominator
        return cbar

    ################################################################################

    """
        Calculate Dj for a trips matrix.
        Two methods are presented here, one which is simple and very slow and one
        which uses python vector maths and is much faster. Once 2 is proven equal
        to 1, then it can be used exclusively. This function is mainly used for
        testing with the TensorFlow and other implementations.
        """

    def calculateDj(self, Tij):
        (M, N) = np.shape(Tij)
        Dj = np.zeros(N)
        Dj = Tij.sum(axis=0)
        return Dj

    ###############################################################################

    """
    run Model run3modes
    Quant model for three modes of transport
    @returns Sij predicted flows between i and j
    """
    def run3modes(self):
        # run model
        # i = employment zone
        # j = residential zone
        # Ei = number of jobs at MSOA location
        # Aj = attractor of HH (number of dwellings)
        # cij_mode = travel cost for "mode" (i.e. road, bus, rail)
        # Modes: Road = 0, Bus = 1, Rail = 2
        # Beta = Beta values for three modes - this is also output
        # QUANT data:
        # Observed trips data: "SObs_1.bin", "SObs_2.bin", "SObs_3.bin"
        # Travel cost per mode: "dis_roads_min.bin", "dis_bus_min.bin", "dis_gbrail_min.bin"
        # Note the use of 1,2,3 for modes in the files different from 0,1,2 in the code.
        # Returns predicted flows per mode: "SPred_1.bin", "SPred_2.bin", "SPred_3.bin"

        # Initialisation of parameters
        converged = False  # initialise convergence check param
        n_modes = 3  # Number of modes
        cij_k = [self.cij_0, self.cij_1, self.cij_2]  # list of cost matrices
        SObs_k = [self.SObs_0, self.SObs_1, self.SObs_2]  # list of obs trips matrices

        # Set up Beta for modes 0, 1 and 2 to 1.0
        Beta = np.ones(n_modes)

        # Compute sum of origins and destinations

        '''
        # OiObs : vector with dimension = number of oringins
        OiObs = np.zeros(self.n)
        for k in range(n_modes):
            OiObs += SObs_k[k].sum(axis=1)
        '''

        # DjObs : vector with dimension = number of destinations
        DjObs = [[] for i in range(n_modes)]
        for k in range(n_modes):
            DjObs[k] = np.zeros(self.n)
        for k in range(n_modes):
            DjObs[k] += SObs_k[k].sum(axis=1)

        DjPred = [[] for i in range(n_modes)]
        DjPredSum = np.zeros(n_modes)
        DjObsSum = np.zeros(n_modes)
        delta = np.zeros(n_modes)


        # Convergence loop:
        print("Calibrating the model...")
        iteration = 0
        while converged != True:
            iteration += 1
            print("Iteration: ", iteration)

            # Initialise variables:
            Sij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty lists equal to n_modes

            # hold copy of pre multiplied copies of -Beta_k * cij[k] for each mode
            ExpMBetaCijk = [[] for k in range(n_modes)]
            for kk in range(n_modes):
                ExpMBetaCijk[kk] = np.exp(-Beta[kk] * cij_k[kk])

            for k in range(n_modes):  # mode loop
                Sij[k] = np.zeros(self.m * self.n).reshape(self.m, self.n)
                for i in range(self.m):
                    denom = 0
                    for kk in range(n_modes):
                        denom += np.sum(self.Aj * ExpMBetaCijk[kk][i, :])
                    Sij2 = self.Ei[i] * (self.Aj * ExpMBetaCijk[k][i, :] / denom)
                    Sij[k][i, :] = Sij2  # put answer slice back in return array

            # Calibration with CBar values
            # Calculate mean predicted trips and mean observed trips (this is CBar)
            CBarPred = np.zeros(n_modes)
            CBarObs = np.zeros(n_modes)
            delta = np.ones(n_modes)
            for k in range(n_modes):
                CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])
                CBarObs[k] = self.computeCBar(SObs_k[k], cij_k[k])
                delta[k] = np.absolute(CBarPred[k] - CBarObs[k])  # the aim is to minimise delta[0]+delta[1]+...
            # delta check on all betas (Beta0, Beta1, Beta2) stopping condition for convergence
            # double gradient descent search on Beta0 and Beta1 and Beta2
            converged = True
            for k in range(n_modes):
                if (delta[k] / CBarObs[k] > 0.001):
                    Beta[k] = Beta[k] * CBarPred[k] / CBarObs[k]
                    converged = False
            '''
            # Calibration with Observed flows
            for k in range(n_modes):
                DjPred[k] = self.calculateDj(Sij[k])
                DjPredSum[k] = np.sum(DjPred[k])
                DjObsSum[k] = np.sum(DjObs[k])
                delta[k] = DjPredSum[k] - DjObsSum[k]
            # delta check on beta stopping condition for convergence
            # gradient descent search on beta
            converged = True
            for k in range(n_modes):
                if (delta[k] / DjObsSum[k] > 0.001):
                    Beta[k] = Beta[k] * DjPredSum[k] / DjObsSum[k]
                    converged = False
            '''
            CBarPred = np.zeros(n_modes)
            # Calculate CBar
            for k in range(n_modes):
                CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])

            # Debug:
            # commuter sum blocks
            TotalSij_roads = Sij[0].sum()
            TotalSij_bus = Sij[1].sum()
            TotalSij_rail = Sij[2].sum()
            TotalEi = self.Ei.sum()  # total jobs = pu+pr above
            # print("i= {0:d} beta_roads={1:.6f} beta_bus={2:.6f} beta_rail={3:.6f} cbar_pred_roads={4:.1f} cbar_pred_busr={5:.1f} cbar_pred_rail={6:.1f}"
            #         .format(iteration, Beta[0], Beta[1], Beta[2], CBarPred[0], CBarPred[1], CBarPred[2]))
            # print("TotalSij_roads={0:.1f} TotalSij_bus={1:.1f} TotalSij_rail={2:.1f} Total={3:.1f} ({4:.1f})"
            #       .format(TotalSij_roads, TotalSij_bus, TotalSij_rail, TotalSij_roads + TotalSij_bus + TotalSij_rail, TotalEi))

        return Sij, Beta, CBarPred  # Note that Sij = [Sij_k=0 , Sij_k=1, Sij_k=2] and CBarPred = [CBarPred_0, CBarPred_1, CBarPred_2]

    ################################################################################

    """
    run Model run3modes_NoCalibration
    Quant model for three modes of transport without calibration
    @returns Sij predicted flows between i and j
    """
    def run3modes_NoCalibration(self, Beta):
        n_modes = len(Beta)  # Number of modes
        print("Running model for ", n_modes, " modes.")

        # Initialise variables:
        Sij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty lists equal to n_modes
        cij_k = [self.cij_0, self.cij_1, self.cij_2]  # list of cost matrices

        # hold copy of pre multiplied copies of -Beta_k * cij[k] for each mode
        ExpMBetaCijk = [[] for k in range(n_modes)]
        for kk in range(n_modes):
            ExpMBetaCijk[kk] = np.exp(-Beta[kk] * cij_k[kk])

        for k in range(n_modes):  # mode loop
            Sij[k] = np.zeros(self.m * self.n).reshape(self.m, self.n)
            for i in range(self.m):
                denom = 0
                for kk in range(n_modes):
                    denom += np.sum(self.Aj * ExpMBetaCijk[kk][i, :])
                Sij2 = self.Ei[i] * (self.Aj * ExpMBetaCijk[k][i, :] / denom)
                Sij[k][i, :] = Sij2  # put answer slice back in return array

        CBarPred = np.zeros(n_modes)  # initialise CBarPred
        for k in range(n_modes):
            CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])

        return Sij, CBarPred

    ################################################################################

    """
    computeProbabilities3modes
    Compute the probability of a flow from an MSOA zone to any (i.e. all) of the possible point zones
    """
    def computeProbabilities3modes(self, Sij):
        print("Computing probabilities")
        n_modes = 3
        probSij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty list equal to n_modes

        for k in range(n_modes):
            probSij[k] = np.arange(self.m * self.n, dtype=np.float64).reshape(self.m, self.n)
            for i in range(self.m):
                sum = np.sum(Sij[k][i,])
                if sum <= 0:
                    sum = 1  # catch for divide by zero - just let the zero probs come through to the final matrix
                probSij[k][i,] = Sij[k][i,] / sum

        return probSij
    
class QUANTRetailModel(QUANTLHModel):
    """
    constructor
    @param n number of residential zones
    @param m number of retail zones
    """
    def __init__(self,m,n):
        #constructor
        super().__init__(m,n)

    ################################################################################

    """
    loadGeolytixData
    @param filename Name of file to load - this is the Geolytix restricted access data with
    the floorspace and retail data
    @returns DataFrame containing [key,zonei,east,north] and [zonei,Modelled turnover annual]
    """
    @staticmethod
    def loadGeolytixData(filename):
        missing_values = ['-', 'n/a', 'na', '--', ' -   ']
        df = pd.read_csv(filename,usecols=['id','fascia','modelled sq ft','Modelled turnover annual','bng_e','bng_n'], na_values=missing_values)
        df.dropna(axis=0,inplace=True)
        df.reset_index(drop=True,inplace=True) # IMPORTANT, otherwise indexes remain for ALL the rows i.e. idx=0..OriginalN NOT true row count!
        dfzones = pd.DataFrame({'id':df.id,'zonei':df.index,'east':df.bng_e,'north':df.bng_n})
        dfattractors = pd.DataFrame({'zonei':df.index,'Modelled turnover annual':df['Modelled turnover annual']}) # could also used floorspace
        return dfzones, dfattractors
    
def costMSOAToPoint_3modes(cij_roads, cij_bus, cij_rail, dfPoints, OXF_MSOA_list):
    # const to define what speed we travel the additional distance to the retail point e.g. 30mph = 13 ms-1
    metresPerSec_roads = 13.0 # 30 miles/h = 47 km/h
    metresPerSec_bus = 13.0  # 30 miles/h = 47 km/h
    metresPerSec_rail = 13.0  # 30 miles/h = 47 km/h

    # read in the road, bus and rail centroids for the cij matrix
    df_roads = pd.read_csv(os.path.join('data', 'roadcentroidlookup_QC.csv'), index_col='zonecode')
    df_bus = pd.read_csv(os.path.join('data', 'buscentroidlookup_QC.csv'), index_col='zonecode')
    df_rail = pd.read_csv(os.path.join('data', 'gbrailcentroidlookup_QC.csv'), index_col='zonecode')

    # Filter out Oxfordshire from EWS files
    df_roads = df_roads.loc[OXF_MSOA_list]  # Filter rows
    df_bus = df_bus.loc[OXF_MSOA_list]  # Filter rows
    df_rail = df_rail.loc[OXF_MSOA_list]  # Filter rows

    df_roads['zonecode'] = df_roads.index  # turn the index (i.e. MSOA codes) back into a columm
    df_bus['zonecode'] = df_bus.index  # turn the index (i.e. MSOA codes) back into a columm
    df_rail['zonecode'] = df_rail.index  # turn the index (i.e. MSOA codes) back into a columm

    # Reset indexes
    df_roads.reset_index(drop=True, inplace=True)
    df_bus.reset_index(drop=True, inplace=True)
    df_rail.reset_index(drop=True, inplace=True)

    # Overwrite the zonei column with the new index
    df_roads['zonei'] = df_roads.index
    df_bus['zonei'] = df_bus.index
    df_rail['zonei'] = df_rail.index

    # code this into a geodataframe so we can make a spatial index
    gdf_roads = gpd.GeoDataFrame(df_roads, crs='epsg:4326', geometry=gpd.points_from_xy(df_roads.vertex_lon, df_roads.vertex_lat))
    gdf_bus = gpd.GeoDataFrame(df_bus, crs='epsg:4326', geometry=gpd.points_from_xy(df_bus.vertex_lon, df_bus.vertex_lat))
    gdf_rail = gpd.GeoDataFrame(df_rail, crs='epsg:4326', geometry=gpd.points_from_xy(df_rail.vertex_lon, df_rail.vertex_lat))

    # but it's lat/lon and we want east/north, convert crs:
    centroids_roads = gdf_roads.to_crs("EPSG:27700")
    centroids_bus = gdf_bus.to_crs("EPSG:27700")
    centroids_rail = gdf_rail.to_crs("EPSG:27700")

    dest_unary_roads = centroids_roads["geometry"].unary_union  # and need this join for the centroid points nearest lookup
    dest_unary_bus = centroids_bus["geometry"].unary_union  # and need this join for the centroid points nearest lookup
    dest_unary_rail = centroids_rail["geometry"].unary_union  # and need this join for the centroid points nearest lookup

    # create a new MSOA to points cost matix
    m, n = cij_roads.shape
    p, cols = dfPoints.shape
    # print("array size = ", p, m)

    cijpoint_roads = np.zeros(m * p, dtype=np.float64).reshape(m, p)  # so m=MSOA and p=points index
    cijpoint_bus = np.zeros(m * p, dtype=np.float64).reshape(m, p)  # so m=MSOA and p=points index
    cijpoint_rail = np.zeros(m * p, dtype=np.float64).reshape(m, p)  # so m=MSOA and p=points index

    # now make the amended cost function
    count = 0
    for row in dfPoints.itertuples(index=False):  # NOTE: iterating over Pandas rows is supposed to be bad - how else to do this?
        if (count % 50 == 0): print("costs::costMSOAToPoint ", count, "/", p)
        count += 1

        p_zonei = getattr(row, 'zonei')
        p_east = getattr(row, 'east')
        p_north = getattr(row, 'north')

        near_roads = nearest_points(Point(p_east, p_north), dest_unary_roads)
        near_bus = nearest_points(Point(p_east, p_north), dest_unary_bus)
        near_rail = nearest_points(Point(p_east, p_north), dest_unary_rail)

        match_geom_roads = centroids_roads.loc[centroids_roads.geometry == near_roads[1]]
        match_geom_bus = centroids_bus.loc[centroids_bus.geometry == near_bus[1]]
        match_geom_rail = centroids_rail.loc[centroids_rail.geometry == near_rail[1]]

        pmsoa_zonei_roads = int(match_geom_roads.zonei.iloc[0])  # closest point msoa zone
        pmsoa_zonei_bus = int(match_geom_bus.zonei.iloc[0])  # closest point msoa zone
        pmsoa_zonei_rail = int(match_geom_rail.zonei.iloc[0])  # closest point msoa zone

        pmsoa_pt_roads = match_geom_roads.geometry
        pmsoa_pt_bus = match_geom_bus.geometry
        pmsoa_pt_rail = match_geom_rail.geometry

        pmsoa_east_roads = float(pmsoa_pt_roads.centroid.x.iloc[0])
        pmsoa_east_bus = float(pmsoa_pt_bus.centroid.x.iloc[0])
        pmsoa_east_rail = float(pmsoa_pt_rail.centroid.x.iloc[0])

        pmsoa_north_roads = float(pmsoa_pt_roads.centroid.y.iloc[0])
        pmsoa_north_bus = float(pmsoa_pt_bus.centroid.y.iloc[0])
        pmsoa_north_rail = float(pmsoa_pt_rail.centroid.y.iloc[0])

        dx_roads = p_east - pmsoa_east_roads
        dx_bus = p_east - pmsoa_east_bus
        dx_rail = p_east - pmsoa_east_rail

        dy_roads = p_north - pmsoa_north_roads
        dy_bus = p_north - pmsoa_north_bus
        dy_rail = p_north - pmsoa_north_rail

        dist_roads = np.sqrt(dx_roads * dx_roads + dy_roads * dy_roads)  # dist between point and centroid used for shortest path
        dist_bus = np.sqrt(dx_bus * dx_bus + dy_bus * dy_bus)  # dist between point and centroid used for shortest path
        dist_rail = np.sqrt(dx_rail * dx_rail + dy_rail * dy_rail)  # dist between point and centroid used for shortest path

        # work out an additional delta cost based on increased time getting from this point to the centroid
        deltaCost_roads = (dist_roads / metresPerSec_roads) / 60.0  # transit time in mins
        deltaCost_bus = (dist_bus / metresPerSec_bus) / 60.0  # transit time in mins
        deltaCost_rail = (dist_rail / metresPerSec_rail) / 60.0  # transit time in mins

        # now write every cij value for msoa_zonei to p_zonei (closest) PLUS deltaCose for p_zonei to actual point
        for i in range(n):
            C1_roads = cij_roads[pmsoa_zonei_roads, i]  # yes, this is right for a trip from MSOA to closest point MSOA - QUANT is BACKWARDS
            C1_bus = cij_bus[pmsoa_zonei_bus, i]  # yes, this is right for a trip from MSOA to closest point MSOA - QUANT is BACKWARDS
            C1_rail = cij_rail[pmsoa_zonei_rail, i]  # yes, this is right for a trip from MSOA to closest point MSOA - QUANT is BACKWARDS

            cijpoint_roads[i, p_zonei] = C1_roads + deltaCost_roads
            cijpoint_bus[i, p_zonei] = C1_bus + deltaCost_bus
            cijpoint_rail[i, p_zonei] = C1_rail + deltaCost_rail

            # NOTE: you can only go in one direction with a matrix that is asymmetric

    return cijpoint_roads, cijpoint_bus, cijpoint_rail

def run3modes_NoCalibration(self, Beta):
        n_modes = len(Beta)  # Number of modes
        print("Running model for ", n_modes, " modes.")

        # Initialise variables:
        Sij = [[] for i in range(n_modes)]  # initialise Sij with a number of empty lists equal to n_modes
        cij_k = [self.cij_0, self.cij_1, self.cij_2]  # list of cost matrices

        # hold copy of pre multiplied copies of -Beta_k * cij[k] for each mode
        ExpMBetaCijk = [[] for k in range(n_modes)]
        for kk in range(n_modes):
            ExpMBetaCijk[kk] = np.exp(-Beta[kk] * cij_k[kk])

        for k in range(n_modes):  # mode loop
            Sij[k] = np.zeros(self.m * self.n).reshape(self.m, self.n)
            for i in range(self.m):
                denom = 0
                for kk in range(n_modes):
                    denom += np.sum(self.Aj * ExpMBetaCijk[kk][i, :])
                Sij2 = self.Ei[i] * (self.Aj * ExpMBetaCijk[k][i, :] / denom)
                Sij[k][i, :] = Sij2  # put answer slice back in return array

        CBarPred = np.zeros(n_modes)  # initialise CBarPred
        for k in range(n_modes):
            CBarPred[k] = self.computeCBar(Sij[k], cij_k[k])

        return Sij, CBarPred

In [4]:
OXF_MSOA_df =  pd.read_excel('data/msoa_data.xlsx', usecols=["msoa11cd"]) # import file with OXF MSOA codes
OXF_MSOA_df.columns = ['areakey'] # rename column "msoa11cd" to "areakey"
OXF_MSOA_list = OXF_MSOA_df['areakey'].tolist()
del OXF_MSOA_list[-1]

zonecodes_EWS = pd.read_csv('data/EWS_ZoneCodes.csv')
zonecodes_EWS.set_index('areakey')
zonecodes_EWS_list = zonecodes_EWS['areakey'].tolist()

cij_road_EWS = loadQUANTMatrix('data/dis_roads_min.bin')
cij_road_EWS_df = pd.DataFrame(cij_road_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list)
cij_road_OXF_df = cij_road_EWS_df[OXF_MSOA_list]
cij_road_OXF_df = cij_road_OXF_df.loc[OXF_MSOA_list]
cij_road_OXF = cij_road_OXF_df.to_numpy()
cij_road_OXF[cij_road_OXF < 1] = 1 

cij_bus_EWS = loadQUANTMatrix('data/dis_bus_min.bin')
cij_bus_EWS_df = pd.DataFrame(cij_bus_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list) # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
cij_bus_OXF_df = cij_bus_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
cij_bus_OXF_df = cij_bus_OXF_df.loc[OXF_MSOA_list]  # Filter rows
cij_bus_OXF = cij_bus_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
cij_bus_OXF[cij_bus_OXF < 1] = 1  # lower limit of 1 minute links

cij_rail_EWS = loadQUANTMatrix('data/dis_gbrail_min.bin')
cij_rail_EWS_df = pd.DataFrame(cij_rail_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list) # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
cij_rail_OXF_df = cij_rail_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
cij_rail_OXF_df = cij_rail_OXF_df.loc[OXF_MSOA_list]  # Filter rows
cij_rail_OXF = cij_rail_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
cij_rail_OXF[cij_rail_OXF < 1] = 1  # lower limit of 1 minute links

SObs_road_EWS = loadQUANTMatrix('data/SObs_1.bin')
SObs_road_EWS_df = pd.DataFrame(SObs_road_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list) # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
SObs_road_OXF_df = SObs_road_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
SObs_road_OXF_df = SObs_road_OXF_df.loc[OXF_MSOA_list]  # Filter rows
SObs_road_OXF = SObs_road_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)

SObs_bus_EWS = loadQUANTMatrix('data/SObs_2.bin')
SObs_bus_EWS_df = pd.DataFrame(SObs_bus_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list)  # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
SObs_bus_OXF_df = SObs_bus_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
SObs_bus_OXF_df = SObs_bus_OXF_df.loc[OXF_MSOA_list]  # Filter rows
SObs_bus_OXF = SObs_bus_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)

SObs_rail_EWS = loadQUANTMatrix('data/SObs_3.bin')
SObs_rail_EWS_df = pd.DataFrame(SObs_rail_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list)  # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
SObs_rail_OXF_df = SObs_rail_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
SObs_rail_OXF_df = SObs_rail_OXF_df.loc[OXF_MSOA_list]  # Filter rows
SObs_rail_OXF = SObs_rail_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)

dfEW = pd.read_csv('data/QS103EW_MSOA.csv')
dfEW['count_allpeople'] = dfEW['Age: All categories: Age; measures: Value'] # you could just rename the col, not copy
dfEW2 = pd.DataFrame({'geography code': dfEW['geography code'], 'count_allpeople': dfEW['count_allpeople']})

dfS = pd.read_csv('data/QS103SC_DZ2001.csv') # join on "Unnamed: 0", it's blank! This is the datazone code field
dfS.set_index('Unnamed: 0')
dfSLookup = pd.read_csv('data/DZ2001Lookup.csv') # join on ZONECODE, which is the datazone code
dfS = dfS.join(other=dfSLookup.set_index('ZONECODE'),on='Unnamed: 0')
dfS['count_allpeople'] = dfS['All people']
dfS2 = dfS.groupby(['IZ_CODE']).agg({'count_allpeople': "sum"})
dfS3 = pd.DataFrame({'msoaiz': dfS2.index, 'count_allpeople': dfS2['count_allpeople'] })

dfEW2.reset_index()
dfS3.reset_index()
dfEW2.columns=['msoaiz','count_allpeople']
dfEWS = pd.concat([dfEW2,dfS3])

dfPopMSOAPopulation_EWS = pd.read_csv('data/totalpopulation_englandwalesscotland_msoaiz.csv', usecols=['msoaiz', 'count_allpeople'], index_col='msoaiz') 

dfPopMSOAPopulation_OXF = dfPopMSOAPopulation_EWS.loc[OXF_MSOA_list]  # Filter rows
dfPopMSOAPopulation_OXF.sort_index(inplace=True)
dfPopMSOAPopulation_OXF['msoaiz'] = dfPopMSOAPopulation_OXF.index  # turn the index (i.e. MSOA codes) back into a columm
dfPopMSOAPopulation_OXF.reset_index(drop=True, inplace=True)  # IMPORTANT, otherwise indexes remain for ALL the rows i.e. idx=0..OriginalN NOT true row count!
popretailPopulation = dfPopMSOAPopulation_OXF.join(other=zonecodes_EWS.set_index('areakey'), on='msoaiz') 

retailpoints_EWS = pd.read_csv('data/geolytix_retailpoints_open_regression.csv')
OXF_postcodes_df = pd.read_excel('data/Oxfordshire_postcodes.xlsx')  # df containing: Postcode,Latitude,Longitude,Eastings,Northings
OXF_postcodes_df['DistrictCode'] = OXF_postcodes_df['pcd'].str[:-3]  # create column with district code
OXF_DistrictCodes = OXF_postcodes_df['DistrictCode'].tolist()  # save the district codes into a list
open_geolitix_OXF = retailpoints_EWS.loc[retailpoints_EWS['postcode'].str[:-4].isin(OXF_DistrictCodes)]  # Filter rows (-4 because of the blank space in the middle of the postcode)
open_geolitix_OXF.reset_index(drop=True, inplace=True)



loadQUANTMatrix::m= 8436 n= 8436
loadQUANTMatrix::m= 8436 n= 8436
loadQUANTMatrix::m= 8436 n= 8436
loadQUANTMatrix::m= 8436 n= 8436
loadQUANTMatrix::m= 8436 n= 8436
loadQUANTMatrix::m= 8436 n= 8436


In [55]:
df = open_geolitix_OXF.sort_values('Modelled turnover annual')

def remover_smallest_10_percentage(group):
    return group.tail(int(0.9 * len(group)))

def remover_smallest_20_percentage(group):
    return group.tail(int(0.8 * len(group)))

def remover_smallest_30_percentage(group):
    return group.tail(int(0.7 * len(group)))

def remover_smallest_40_percentage(group):
    return group.tail(int(0.6 * len(group)))

def remover_smallest_50_percentage(group):
    return group.tail(int(0.5 * len(group)))

df_10 = df.groupby('modelled sq ft').apply(remover_smallest_10_percentage).reset_index(drop=True)
df_20 = df.groupby('modelled sq ft').apply(remover_smallest_20_percentage).reset_index(drop=True)
df_30 = df.groupby('modelled sq ft').apply(remover_smallest_30_percentage).reset_index(drop=True)
df_40 = df.groupby('modelled sq ft').apply(remover_smallest_40_percentage).reset_index(drop=True)
df_50 = df.groupby('modelled sq ft').apply(remover_smallest_50_percentage).reset_index(drop=True)
df_10.to_csv('data/geolytix_retailpoints_open_regression_OXF_10.csv')
df_20.to_csv('data/geolytix_retailpoints_open_regression_OXF_20.csv')
df_30.to_csv('data/geolytix_retailpoints_open_regression_OXF_30.csv')
df_40.to_csv('data/geolytix_retailpoints_open_regression_OXF_40.csv')
df_50.to_csv('data/geolytix_retailpoints_open_regression_OXF_50.csv')

In [56]:
popretailZones, popretailAttractors = QUANTRetailModel.loadGeolytixData('data/geolytix_retailpoints_open_regression_OXF.csv')
popretailZones_10, popretailAttractors_10 = QUANTRetailModel.loadGeolytixData('data/geolytix_retailpoints_open_regression_OXF_10.csv')
popretailZones_20, popretailAttractors_20 = QUANTRetailModel.loadGeolytixData('data/geolytix_retailpoints_open_regression_OXF_20.csv')
popretailZones_30, popretailAttractors_30 = QUANTRetailModel.loadGeolytixData('data/geolytix_retailpoints_open_regression_OXF_30.csv')
popretailZones_40, popretailAttractors_40 = QUANTRetailModel.loadGeolytixData('data/geolytix_retailpoints_open_regression_OXF_40.csv')
popretailZones_50, popretailAttractors_50 = QUANTRetailModel.loadGeolytixData('data/geolytix_retailpoints_open_regression_OXF_50.csv')



In [57]:
retailpoints_cij_roads, retailpoints_cij_bus, retailpoints_cij_rail = costMSOAToPoint_3modes(cij_road_OXF, cij_bus_OXF, cij_rail_OXF, popretailZones, OXF_MSOA_list)
retailpoints_cij_roads_10, retailpoints_cij_bus_10, retailpoints_cij_rail_10 = costMSOAToPoint_3modes(cij_road_OXF, cij_bus_OXF, cij_rail_OXF, popretailZones_10, OXF_MSOA_list)
retailpoints_cij_roads_20, retailpoints_cij_bus_20, retailpoints_cij_rail_20 = costMSOAToPoint_3modes(cij_road_OXF, cij_bus_OXF, cij_rail_OXF, popretailZones_20, OXF_MSOA_list)
retailpoints_cij_roads_30, retailpoints_cij_bus_30, retailpoints_cij_rail_30 = costMSOAToPoint_3modes(cij_road_OXF, cij_bus_OXF, cij_rail_OXF, popretailZones_30, OXF_MSOA_list)
retailpoints_cij_roads_40, retailpoints_cij_bus_40, retailpoints_cij_rail_40 = costMSOAToPoint_3modes(cij_road_OXF, cij_bus_OXF, cij_rail_OXF, popretailZones_40, OXF_MSOA_list)
retailpoints_cij_roads_50, retailpoints_cij_bus_50, retailpoints_cij_rail_50 = costMSOAToPoint_3modes(cij_road_OXF, cij_bus_OXF, cij_rail_OXF, popretailZones_50, OXF_MSOA_list)

costs::costMSOAToPoint  0 / 520
costs::costMSOAToPoint  50 / 520
costs::costMSOAToPoint  100 / 520
costs::costMSOAToPoint  150 / 520
costs::costMSOAToPoint  200 / 520
costs::costMSOAToPoint  250 / 520
costs::costMSOAToPoint  300 / 520
costs::costMSOAToPoint  350 / 520
costs::costMSOAToPoint  400 / 520
costs::costMSOAToPoint  450 / 520
costs::costMSOAToPoint  500 / 520
costs::costMSOAToPoint  0 / 466
costs::costMSOAToPoint  50 / 466
costs::costMSOAToPoint  100 / 466
costs::costMSOAToPoint  150 / 466
costs::costMSOAToPoint  200 / 466
costs::costMSOAToPoint  250 / 466
costs::costMSOAToPoint  300 / 466
costs::costMSOAToPoint  350 / 466
costs::costMSOAToPoint  400 / 466
costs::costMSOAToPoint  450 / 466
costs::costMSOAToPoint  0 / 414
costs::costMSOAToPoint  50 / 414
costs::costMSOAToPoint  100 / 414
costs::costMSOAToPoint  150 / 414
costs::costMSOAToPoint  200 / 414
costs::costMSOAToPoint  250 / 414
costs::costMSOAToPoint  300 / 414
costs::costMSOAToPoint  350 / 414
costs::costMSOAToPoint 

In [31]:
m, n = retailpoints_cij_roads.shape
model = QUANTRetailModel(m,n)
model.setAttractorsAj(popretailAttractors,'zonei','Modelled turnover annual')
model.setPopulationEi(popretailPopulation,'zonei','count_allpeople')
model.setCostMatrixCij(retailpoints_cij_roads, retailpoints_cij_bus, retailpoints_cij_rail)
beta = np.array([0.3127422 , 0.07877828, 0.05523665]) # from the Journey to work model calibration
Rij, cbar = model.run3modes_NoCalibration(beta)
popretail_probRij = model.computeProbabilities3modes(Rij)

Running model for  3  modes.
Computing probabilities


In [32]:
m, n = retailpoints_cij_roads_10.shape
model = QUANTRetailModel(m,n)
model.setAttractorsAj(popretailAttractors_10,'zonei','Modelled turnover annual')
model.setPopulationEi(popretailPopulation,'zonei','count_allpeople')
model.setCostMatrixCij(retailpoints_cij_roads_10, retailpoints_cij_bus_10, retailpoints_cij_rail_10)
beta = np.array([0.3127422 , 0.07877828, 0.05523665]) # from the Journey to work model calibration
Rij_10, cbar_10 = model.run3modes_NoCalibration(beta)
popretail_probRij_10 = model.computeProbabilities3modes(Rij_10)

Running model for  3  modes.
Computing probabilities


In [58]:
m, n = retailpoints_cij_roads_20.shape
model = QUANTRetailModel(m,n)
model.setAttractorsAj(popretailAttractors_20,'zonei','Modelled turnover annual')
model.setPopulationEi(popretailPopulation,'zonei','count_allpeople')
model.setCostMatrixCij(retailpoints_cij_roads_20, retailpoints_cij_bus_20, retailpoints_cij_rail_20)
beta = np.array([0.3127422 , 0.07877828, 0.05523665]) # from the Journey to work model calibration
Rij_20, cbar_20 = model.run3modes_NoCalibration(beta)
popretail_probRij_20 = model.computeProbabilities3modes(Rij_20)

Running model for  3  modes.
Computing probabilities


In [33]:
m, n = retailpoints_cij_roads_30.shape
model = QUANTRetailModel(m,n)
model.setAttractorsAj(popretailAttractors_30,'zonei','Modelled turnover annual')
model.setPopulationEi(popretailPopulation,'zonei','count_allpeople')
model.setCostMatrixCij(retailpoints_cij_roads_30, retailpoints_cij_bus_30, retailpoints_cij_rail_30)
beta = np.array([0.3127422 , 0.07877828, 0.05523665]) # from the Journey to work model calibration
Rij_30, cbar_30 = model.run3modes_NoCalibration(beta)
popretail_probRij_30 = model.computeProbabilities3modes(Rij_30)

Running model for  3  modes.
Computing probabilities


In [59]:
m, n = retailpoints_cij_roads_40.shape
model = QUANTRetailModel(m,n)
model.setAttractorsAj(popretailAttractors_40,'zonei','Modelled turnover annual')
model.setPopulationEi(popretailPopulation,'zonei','count_allpeople')
model.setCostMatrixCij(retailpoints_cij_roads_40, retailpoints_cij_bus_40, retailpoints_cij_rail_40)
beta = np.array([0.3127422 , 0.07877828, 0.05523665]) # from the Journey to work model calibration
Rij_40, cbar_40 = model.run3modes_NoCalibration(beta)
popretail_probRij_40 = model.computeProbabilities3modes(Rij_40)

Running model for  3  modes.
Computing probabilities


In [60]:
m, n = retailpoints_cij_roads_50.shape
model = QUANTRetailModel(m,n)
model.setAttractorsAj(popretailAttractors_50,'zonei','Modelled turnover annual')
model.setPopulationEi(popretailPopulation,'zonei','count_allpeople')
model.setCostMatrixCij(retailpoints_cij_roads_50, retailpoints_cij_bus_50, retailpoints_cij_rail_50)
beta = np.array([0.3127422 , 0.07877828, 0.05523665]) # from the Journey to work model calibration
Rij_50, cbar_50 = model.run3modes_NoCalibration(beta)
popretail_probRij_50 = model.computeProbabilities3modes(Rij_50)

Running model for  3  modes.
Computing probabilities


In [61]:
np.savetxt('data/populationretailProbRij_roads_2011_10.csv', popretail_probRij_10[0], delimiter=",")
np.savetxt('data/populationretailProbRij_bus_2011_10.csv', popretail_probRij_10[1], delimiter=",")
np.savetxt('data/populationretailProbRij_rail_2011_10.csv', popretail_probRij_10[2], delimiter=",")
np.savetxt('data/populationretailRij_roads_2011_10.csv', Rij_10[0], delimiter=",")
np.savetxt('data/populationretailRij_bus_2011_10.csv', Rij_10[1], delimiter=",")
np.savetxt('data/populationretailRij_rail_2011_10.csv', Rij_10[2], delimiter=",")

np.savetxt('data/populationretailProbRij_roads_2011_20.csv', popretail_probRij_20[0], delimiter=",")
np.savetxt('data/populationretailProbRij_bus_2011_20.csv', popretail_probRij_20[1], delimiter=",")
np.savetxt('data/populationretailProbRij_rail_2011_20.csv', popretail_probRij_20[2], delimiter=",")
np.savetxt('data/populationretailRij_roads_2011_20.csv', Rij_20[0], delimiter=",")
np.savetxt('data/populationretailRij_bus_2011_20.csv', Rij_20[1], delimiter=",")
np.savetxt('data/populationretailRij_rail_2011_20.csv', Rij_30[2], delimiter=",")

np.savetxt('data/populationretailProbRij_roads_2011_30.csv', popretail_probRij_30[0], delimiter=",")
np.savetxt('data/populationretailProbRij_bus_2011_30.csv', popretail_probRij_30[1], delimiter=",")
np.savetxt('data/populationretailProbRij_rail_2011_30.csv', popretail_probRij_30[2], delimiter=",")
np.savetxt('data/populationretailRij_roads_2011_30.csv', Rij_30[0], delimiter=",")
np.savetxt('data/populationretailRij_bus_2011_30.csv', Rij_30[1], delimiter=",")
np.savetxt('data/populationretailRij_rail_2011_30.csv', Rij_30[2], delimiter=",")

np.savetxt('data/populationretailProbRij_roads_2011_40.csv', popretail_probRij_40[0], delimiter=",")
np.savetxt('data/populationretailProbRij_bus_2011_40.csv', popretail_probRij_40[1], delimiter=",")
np.savetxt('data/populationretailProbRij_rail_2011_40.csv', popretail_probRij_40[2], delimiter=",")
np.savetxt('data/populationretailRij_roads_2011_40.csv', Rij_40[0], delimiter=",")
np.savetxt('data/populationretailRij_bus_2011_40.csv', Rij_40[1], delimiter=",")
np.savetxt('data/populationretailRij_rail_2011_40.csv', Rij_40[2], delimiter=",")

np.savetxt('data/populationretailProbRij_roads_2011_50.csv', popretail_probRij_50[0], delimiter=",")
np.savetxt('data/populationretailProbRij_bus_2011_50.csv', popretail_probRij_50[1], delimiter=",")
np.savetxt('data/populationretailProbRij_rail_2011_50.csv', popretail_probRij_50[2], delimiter=",")
np.savetxt('data/populationretailRij_roads_2011_50.csv', Rij_50[0], delimiter=",")
np.savetxt('data/populationretailRij_bus_2011_50.csv', Rij_50[1], delimiter=",")
np.savetxt('data/populationretailRij_rail_2011_50.csv', Rij_50[2], delimiter=",")

In [28]:
gdf = gpd.read_file('data/ESRI/MSOA_2011_London_gen_MHW.shp')
gdf = gdf.to_crs('EPSG: 4326')
gdf['geometry'] = gdf['geometry'].centroid
gdf = gdf[['geometry']]

def calc_shortest_paths_ODs_osm(zones_centroids, network):
    list_of_ODs = []
    for c in zones_centroids.geometry:
        graph_clostest_node = ox.nearest_nodes(network, c.x, c.y, return_dist=False)
        list_of_ODs.append(graph_clostest_node)
    return list_of_ODs


/var/folders/63/l2ny8d3j1rl_k0h8ghjdkflm0000gn/T/ipykernel_3219/1128587817.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].centroid


In [29]:
Zone_nodes = gpd.read_file('data/OXF_MSOA_centroids_WGS84.shp')
Case_Study_Zones = ['London, UK']
    # Case_Study_Zones = ['Oxford'] # Smaller network for test purposes

X = ox.graph_from_place(Case_Study_Zones, network_type='drive')
X = X.to_undirected()

In [30]:
def calc_shortest_paths_ODs_osm(zones_centroids, network):
    list_of_ODs = []
    for c in zones_centroids.geometry:
        graph_clostest_node = ox.nearest_nodes(network, c.x, c.y, return_dist=False)
        list_of_ODs.append(graph_clostest_node)
    return list_of_ODs

# Convert GeoDataFrame to suitable format for calc_shortest_paths_ODs_osm function
OD_list = calc_shortest_paths_ODs_osm(Zone_nodes, X)

In [36]:
Flows = []
flows_output_keys = ['data/populationretailRij_roads_2011_10.csv',
                     'data/populationretailRij_bus_2011_10.csv',
                     'data/populationretailRij_roads_2011_30.csv', 
                     'data/populationretailRij_bus_2011_30.csv']

for kk, flows_output_key in enumerate(flows_output_keys):
        Flows.append(pd.read_csv(flows_output_key, header=None))

        # Initialise weights to 0:
        for source, target in X.edges():
            X[source][target][0]["Flows_" + str(kk)] = 0

TOT_count = len(OD_list)
    # print(OD_list)

In [47]:
for n, i in enumerate(OD_list):
    print("Flows maps creation - iteration ", n+1, " of ", TOT_count)
    sssp_paths = nx.single_source_dijkstra_path(X, i, weight='length') 
    for m, j in enumerate(OD_list):
        for cc in range(len(Flows)):
            if m >= Flows[cc].shape[1]:
                continue  # Skip if m exceeds the number of columns in Flows[cc]
            
            shortest_path = sssp_paths[j] 
            path_edges = zip(shortest_path, shortest_path[1:])

            for edge in list(path_edges):
                X[edge[0]][edge[1]][0]["Flows_" + str(cc)] += Flows[cc].iloc[n, m]

Flows maps creation - iteration  1  of  983
Flows maps creation - iteration  2  of  983
Flows maps creation - iteration  3  of  983
Flows maps creation - iteration  4  of  983
Flows maps creation - iteration  5  of  983
Flows maps creation - iteration  6  of  983
Flows maps creation - iteration  7  of  983
Flows maps creation - iteration  8  of  983
Flows maps creation - iteration  9  of  983
Flows maps creation - iteration  10  of  983
Flows maps creation - iteration  11  of  983
Flows maps creation - iteration  12  of  983
Flows maps creation - iteration  13  of  983
Flows maps creation - iteration  14  of  983
Flows maps creation - iteration  15  of  983
Flows maps creation - iteration  16  of  983
Flows maps creation - iteration  17  of  983
Flows maps creation - iteration  18  of  983
Flows maps creation - iteration  19  of  983
Flows maps creation - iteration  20  of  983
Flows maps creation - iteration  21  of  983
Flows maps creation - iteration  22  of  983
Flows maps creation

In [48]:
X

In [49]:
ox.save_graph_shapefile(X, filepath='Flows_shp')

/Users/pinlyu/opt/anaconda3/lib/python3.9/site-packages/osmnx/io.py:87: UserWarning: The `save_graph_shapefile` function is deprecated and will be removed in a future release. Instead, use the `save_graph_geopackage` function to save graphs as GeoPackage files for subsequent GIS analysis.
  warnings.warn(
/Users/pinlyu/opt/anaconda3/lib/python3.9/site-packages/osmnx/io.py:114: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nodes.to_file(filepath_nodes, driver="ESRI Shapefile", index=True, encoding=encoding)


In [50]:
ox.save_graph_geopackage(X, filepath='Flows.gpkg')

In [62]:
gdf1 = gpd.read_file('data/ESRI/MSOA_2011_London_gen_MHW.shp')
gdf1 = gdf1.to_crs('EPSG: 4326')
gdf1['geometry'] = gdf1['geometry'].centroid
gdf1 = gdf1[['geometry']]

def calc_shortest_paths_ODs_osm(zones_centroids, network):
    list_of_ODs = []
    for c in zones_centroids.geometry:
        graph_clostest_node = ox.nearest_nodes(network, c.x, c.y, return_dist=False)
        list_of_ODs.append(graph_clostest_node)
    return list_of_ODs

Zone_nodes1 = gpd.read_file('data/OXF_MSOA_centroids_WGS84.shp')
Case_Study_Zones1 = ['London, UK']
    # Case_Study_Zones = ['Oxford'] # Smaller network for test purposes

X1 = ox.graph_from_place(Case_Study_Zones1, network_type='drive')
X1 = X1.to_undirected()

def calc_shortest_paths_ODs_osm(zones_centroids, network):
    list_of_ODs = []
    for c in zones_centroids.geometry:
        graph_clostest_node = ox.nearest_nodes(network, c.x, c.y, return_dist=False)
        list_of_ODs.append(graph_clostest_node)
    return list_of_ODs

# Convert GeoDataFrame to suitable format for calc_shortest_paths_ODs_osm function
OD_list1= calc_shortest_paths_ODs_osm(Zone_nodes1, X1)

Flows1 = []
flows_output_keys1 = ['data/populationretailRij_roads_2011_10.csv',
                     'data/populationretailRij_bus_2011_10.csv',
                     'data/populationretailRij_roads_2011_20.csv',
                     'data/populationretailRij_bus_2011_20.csv',
                     'data/populationretailRij_roads_2011_30.csv', 
                     'data/populationretailRij_bus_2011_30.csv',
                     'data/populationretailRij_roads_2011_40.csv',
                     'data/populationretailRij_bus_2011_40.csv',
                     'data/populationretailRij_roads_2011_50.csv',
                     'data/populationretailRij_bus_2011_50.csv',]

for kk, flows_output_key in enumerate(flows_output_keys1):
        Flows1.append(pd.read_csv(flows_output_key, header=None))

        # Initialise weights to 0:
        for source, target in X1.edges():
            X1[source][target][0]["Flows_" + str(kk)] = 0

TOT_count1 = len(OD_list1)
    # print(OD_list)

for n, i in enumerate(OD_list1):
    print("Flows maps creation - iteration ", n+1, " of ", TOT_count1)
    sssp_paths1 = nx.single_source_dijkstra_path(X1, i, weight='length') 
    for m, j in enumerate(OD_list1):
        for cc in range(len(Flows1)):
            if m >= Flows1[cc].shape[1]:
                continue  # Skip if m exceeds the number of columns in Flows[cc]
            
            shortest_path1= sssp_paths1[j] 
            path_edges1 = zip(shortest_path1, shortest_path1[1:])

            for edge in list(path_edges1):
                X1[edge[0]][edge[1]][0]["Flows_" + str(cc)] += Flows1[cc].iloc[n, m]

/var/folders/63/l2ny8d3j1rl_k0h8ghjdkflm0000gn/T/ipykernel_3219/157862635.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf1['geometry'] = gdf1['geometry'].centroid


Flows maps creation - iteration  1  of  983
Flows maps creation - iteration  2  of  983
Flows maps creation - iteration  3  of  983
Flows maps creation - iteration  4  of  983
Flows maps creation - iteration  5  of  983
Flows maps creation - iteration  6  of  983
Flows maps creation - iteration  7  of  983
Flows maps creation - iteration  8  of  983
Flows maps creation - iteration  9  of  983
Flows maps creation - iteration  10  of  983
Flows maps creation - iteration  11  of  983
Flows maps creation - iteration  12  of  983
Flows maps creation - iteration  13  of  983
Flows maps creation - iteration  14  of  983
Flows maps creation - iteration  15  of  983
Flows maps creation - iteration  16  of  983
Flows maps creation - iteration  17  of  983
Flows maps creation - iteration  18  of  983
Flows maps creation - iteration  19  of  983
Flows maps creation - iteration  20  of  983
Flows maps creation - iteration  21  of  983
Flows maps creation - iteration  22  of  983
Flows maps creation

In [63]:
ox.save_graph_geopackage(X1, filepath='Flows1.gpkg')